In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## results = Loss: 0.8320606350898743 
Acc:0.7473118305206299 
F1 score:0.6648995876312256  
Precision: 0.859656035900116
Recall: 0.5507643818855286

In [ ]:
# load the dataset
from pandas import read_csv
from scipy.special import expit
from sklearn.utils import shuffle
import pickle
def load_dataset_train(full_path = '/content/drive/MyDrive/Privacy/real_data_train.csv'):
  # load the dataset as a numpy array
  with open(full_path, 'rb') as f:
    data = pickle.load(f)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X_train, y_train = load_dataset_train()
X_train = expit(X_train)
print(X_train.shape, y_train.shape)
print(X_train[0])


def load_dataset_test(full_path = '/content/drive/MyDrive/Privacy/real_data_test.csv'):
  # load the dataset as a numpy array
  with open(full_path, 'rb') as f:
    data = pickle.load(f)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X_test, y_test = load_dataset_test()
X_test = expit(X_test)
print(X_test.shape)

(1000, 4) (1000,)
[0.99411941 0.99982842 0.05258531 0.18114637]
(372, 4)


# DPGAN

In [ ]:
from pandas import read_csv
import numpy as np
dataframe = read_csv('/content/drive/MyDrive/Privacy/DPGAN_neg_NEWdata.csv', header=None)
data = dataframe.values
# split into input and output elements
x_test_neg = data[:,1:]
print(x_test_neg[3])
y_test_neg = np.zeros(x_test_neg.shape[0])
print(y_test_neg.shape)


from pandas import read_csv
dataframe2 = read_csv("/content/drive/MyDrive/Privacy/DPGAN_pos_NEWdata.csv", header=None)
data2 = dataframe2.values
# print(data2.shape)
x_test_pos = data2[:,1:]
print(x_test_pos[10])
y_test_pos = np.ones(x_test_pos.shape[0])
print(y_test_pos.shape)



# DPGAN train data
from sklearn.model_selection import train_test_split
x_s_dpgan = np.concatenate((x_test_neg[:,:], x_test_pos))
y_s_dpgan = np.concatenate((y_test_neg[:], y_test_pos))
# X_train_dpgan, _, Y_train_dpgan, _ = train_test_split(x_s_dpgan, y_s_dpgan, test_size = 0.001, random_state = 0)
X_train_dpgan , Y_train_dpgan = shuffle(x_s_dpgan,y_s_dpgan)
print(X_train_dpgan.shape)
print(Y_train_dpgan.shape)

[0.8417605  0.67365986 0.8866259  0.50857836]
(501,)
[-0.2195718   0.77580434  0.46309352  0.4026301 ]
(501,)
(1002, 4)
(1002,)


# model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
import tensorflow as tf



def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


model = Sequential([
Dense(input_dim = 4, units = 2, activation = 'relu'),
# Dense(units = 32, activation = 'relu'),
# Dropout(0.2),
Dense(units = 10, activation = 'relu'),
Dense(units = 4, activation = 'relu'),
Dense(units =1, activation = 'sigmoid'),])


In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', f1_m, precision_m, recall_m])
model.fit(X_train_dpgan, Y_train_dpgan, batch_size = 15, epochs = 15)
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print(loss, accuracy, f1_score, precision, recall)

Epoch 1/15
67/67 [==============================] - 1s 1ms/step - loss: 0.0023 - accuracy: 0.9974 - f1_m: 0.9964 - precision_m: 0.9942 - recall_m: 0.9993
Epoch 2/15
67/67 [==============================] - 0s 1ms/step - loss: 0.0020 - accuracy: 0.9994 - f1_m: 0.9995 - precision_m: 0.9991 - recall_m: 1.0000
Epoch 3/15
67/67 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 0.9987 - f1_m: 0.9986 - precision_m: 0.9983 - recall_m: 0.9992
Epoch 4/15
67/67 [==============================] - 0s 1ms/step - loss: 0.0025 - accuracy: 0.9970 - f1_m: 0.9975 - precision_m: 0.9967 - recall_m: 0.9986
Epoch 5/15
67/67 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 0.9989 - f1_m: 0.9990 - precision_m: 0.9982 - recall_m: 1.0000
Epoch 6/15
67/67 [==============================] - 0s 1ms/step - loss: 0.0022 - accuracy: 0.9992 - f1_m: 0.9991 - precision_m: 0.9984 - recall_m: 1.0000
Epoch 7/15
67/67 [==============================] - 0s 1ms/step - loss: 0.00